In [117]:
import warnings

import numpy as np
import pandas as pd

from itertools import product
from perlin_numpy import generate_perlin_noise_2d
from scipy.interpolate import griddata
from scipy.stats import percentileofscore
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [118]:
np.random.seed(1)

obj = "crosslegtable"

m = 5
n = 100

percentile = 50

iterations = 10

laplace_alpha = 0

degree = 2

In [119]:
ackley = lambda X, Y: -(20 * (1 - np.exp(-0.2 * np.sqrt(0.5 * (X**2 + Y**2)))) - np.exp(0.5 * (np.cos(2 * np.pi * X) + np.cos(2 * np. pi * Y))) + np.exp(1))
beale = lambda X, Y: -((1.5 - X + X * Y)**2 + (2.25 - X + X * Y**2)**2 + (2.625 - X + X * Y**3)**2)
bird = lambda X, Y: -(np.sin(X) * np.exp((1 - np.cos(Y))**2) + np.cos(Y) * np.exp((1 - np.sin(X))**2) + (X - Y)**2)
booth = lambda X, Y: -((X + 2 * Y - 7)**2 + (2 * X + Y - 5)**2)
bukin4 = lambda X, Y: -(100 * Y**2 + 0.01 * np.abs(X + 10))
bukin6 = lambda X, Y: -(100 * np.sqrt(abs(Y - 0.01 * X**2)) + 0.01 * np.abs(X + 10))
carromtable = lambda X, Y: ((np.cos(X) * np.cos(Y) * np.exp(np.abs(1 - np.sqrt(X ** 2 + Y ** 2) / np.pi)))**2) / 30
chichinadze = lambda X, Y: -(X**2 - 12 * X + 11 + 10 * np.cos(np.pi * X / 2) + 8 * np.sin(5 * np.pi * X / 2) - 1 / np.sqrt(5) * np.exp(-((Y - 0.5)**2) / 2))
crossfunc = lambda X, Y: -(np.abs(np.sin(X) * np.sin(Y) * np.exp(np.abs(100 - np.sqrt(X**2 + Y**2) / np.pi))) + 1)**-0.1
crossintray = lambda X, Y: 0.0001 * (np.abs(np.sin(X) * np.sin(Y) * np.exp(np.abs(100 - np.sqrt(X**2 + Y**2) / np.pi))) + 1)**0.1
crosslegtable = lambda X, Y: (np.abs(np.sin(X) * np.sin(Y) * np.exp(np.abs(100 - np.sqrt(X**2 + Y**2) / np.pi))) + 1)**-0.1
crownedcross = lambda X, Y: -0.0001 * (np.abs(np.sin(X) * np.sin(Y) * np.exp(np.abs(100 - np.sqrt(X**2 + Y**2) / np.pi))) + 1)**0.1
cube = lambda X, Y: np.tile(-np.sum(100 * (Y - X**3)**2 + (1 - X)**2, axis=0), (X.shape[0], 1))
easom = lambda X, Y: np.cos(X) * np.cos(Y) * np.exp(-((X - np.pi)**2 + (Y - np.pi)**2))
eggholder = lambda X, Y: np.tile(-np.sum(-(Y + 47) * np.sin(np.sqrt(np.abs(Y + X / 2 + 47))) - X * np.sin(np.sqrt(np.abs(X - (Y + 47)))), axis=0), (X.shape[0], 1))
giunta = lambda X, Y: np.tile(-(0.6 + np.sum(np.sin(16 * (X / 15 - 1) + Y) + np.sin(16 * (X / 15 - 1) + Y)**2 + np.sin(4 * 16 * (X / 15 - 1) + Y) / 50, axis=0)), (X.shape[0], 1))
goldsteinprice = lambda X, Y: -(1 + (X + Y + 1)**2 * (19 - 14 * X + 3 * X**2 - 14 * Y + 6 * X * Y + 3 * Y**2)) * (30 + (2 * X - 3 * Y)**2 * (18 - 32 * X + 12 * X**2 + 48 * Y - 36 * X * Y + 27 * Y**2))
griewank = lambda X, Y: -((X**2 + Y**2) / 200 - np.cos(X) * np.cos(Y / np.sqrt(2)) + 1)
himmelblau = lambda X, Y: -((X**2 + Y - 11)**2 + (X + Y**2 - 7)**2)
holdertable = lambda X, Y: np.abs(np.sin(X) * np.cos(Y) * np.exp(np.abs(1 - np.sqrt(X**2 + Y**2) / np.pi)))
leon = lambda X, Y: -(100 * (Y - X**3)**2 + (1 - X)**2)
levi13 = lambda X, Y: -(np.sin(3 * np.pi * X)**2 + (X - 1)**2 * (1 + np.sin(3 * np.pi * Y)**2) + (Y - 1)**2 * (1 + np.sin(2 * np.pi * Y)**2))
matyas = lambda X, Y: -(0.26 * (X**2 + Y**2) - 0.48 * X * Y)
mccormick = lambda X, Y: -(np.sin(X + Y) + (X - Y)**2 - 1.5 * X + 2.5 * Y + 1)
modschaffer1 = lambda X, Y: -(0.5 + (np.sin(X**2 + Y**2)**2 - 0.5) / (1 + 0.001 * (X**2 + Y**2))**2)
modschaffer2 = lambda X, Y: -(0.5 + (np.sin(X**2 - Y**2)**2 - 0.5) / (1 + 0.001 * (X**2 + Y**2))**2)
modschaffer3 = lambda X, Y: -(0.5 + (np.sin(np.cos(np.abs(X**2 - Y**2)))**2 - 0.5) / (1 + 0.001 * (X**2 + Y**2))**2)
modschaffer4 = lambda X, Y: -(0.5 + (np.cos(np.sin(np.abs(X**2 - Y**2)))**2 - 0.5) / (1 + 0.001 * (X**2 + Y**2))**2)
penholder = lambda X, Y: np.exp(-(np.abs(np.cos(X) * np.cos(Y) * np.exp(np.abs(1 - np.sqrt(X**2 + Y**2) / np.pi))))**-1)
perlin = lambda X, Y: generate_perlin_noise_2d((X.shape[0], X.shape[1]), (2, 2))
rastrigin = lambda X, Y: -(X**2 + Y**2 - 10 * np.cos(2 * np.pi * X) - 10 * np.cos(2 * np.pi * Y) + 2)
rosenbrock = lambda X, Y: np.tile(-np.sum(100 * (Y - X**2)**2 + (1 - X)**2, axis=0), (X.shape[0], 1))
schweffel = lambda X, Y: X * np.sin(np.sqrt(np.abs(X))) + Y * np.sin(np.sqrt(np.abs(Y)))
sinenvsin = lambda X, Y: np.tile(-np.sum((np.sin(np.sqrt(X**2 + Y**2))**2 - 0.5) / (1 + 0.001 * (X**2 + Y**2))**2 + 0.5, axis=0), (X.shape[0], 1))
sixhumpcamel = lambda X, Y: -((4 - 2.1 * X**2 + X**4 / 3) * X**2 + X * Y + (4 * Y**2 - 4) * Y**2)
testtubeholder = lambda X, Y: 4 * np.abs(np.sin(X) * np.cos(Y) * np.exp(np.abs(np.cos((X**2 + Y**2) / 200))))
threehumpcamel = lambda X, Y: -(2 * X**2 - 1.05 * X**4 + X**6 / 6 + X * Y + Y**2)
zettl = lambda X, Y: -((X**2 + Y**2 - 2 * X)**2 + X / 4)

def gaussian_min_max(X, Y):
    Z = np.zeros_like(X)

    for i, j in np.ndindex(Z.shape):
        Z[i, j] = np.min(np.exp(-(X[i, j]**2 + Y[i, j]**2 + np.linspace(-1, 1, 100)[:, np.newaxis]**2) / 2) / np.sqrt(2 / np.pi)**3)

    return Z

def helicalvally_min_max(X, Y):
    Z = np.zeros_like(X)

    for i, j in np.ndindex(Z.shape):
        Z[i, j] = np.min(-(100 * ((Z[i, j] - 10 * np.arctan2(Y[i, j], X[i, j]) / 2 / np.pi)**2 + (np.sqrt(X[i, j]**2 + Y[i, j]**2) - 1)**2) + Z[i, j]**2))

    return Z

def logarithmic_min_max(X, Y):
    Z = np.zeros_like(X)

    for i, j in np.ndindex(Z.shape):
        Z[i, j] = X[i, j]**2 * np.log((X[i, j] - Y[i, j] + Z[i, j])**2 + 1)

    return Z

objectives = {
    "ackley": {
        "func": ackley,
        "bounds": [(-35, 35)] * 2
    },
    "beale": {
        "func": beale,
        "bounds": [(-4.5, 4.5)] * 2
    },
    "bird": {
        "func": bird,
        "bounds": [(-2 * np.pi, 2 * np.pi)] * 2
    },
    "booth": {
        "func": booth,
        "bounds": [(-10, 10)] * 2
    },
    "bukin4": {
        "func": bukin4,
        "bounds": [(-15, 5), (-3, 3)]
    },
    "bukin6": {
        "func": bukin6,
        "bounds": [(-15, 5), (-3, 3)]
    },
    "carromtable": {
        "func": carromtable,
        "bounds": [(-10, 10)] * 2
    },
    "chichinadze": {
        "func": chichinadze,
        "bounds": [(-30, 30)] * 2
    },
    "crossfunc": {
        "func": crossfunc,
        "bounds": [(-10, 10)] * 2
    },
    "crossintray": {
        "func": crossintray,
        "bounds": [(-10, 10)] * 2
    },
    "crosslegtable": {
        "func": crosslegtable,
        "bounds": [(-10, 10)] * 2
    },
    "crownedcross": {
        "func": crownedcross,
        "bounds": [(-10, 10)] * 2
    },
    "cube": {
        "func": cube,
        "bounds": [(-5, 5)] * 2
    },
    "easom": {
        "func": easom,
        "bounds": [(-100, 100)] * 2
    },
    "eggholder": {
        "func": eggholder,
        "bounds": [(-512, 512)] * 2
    },
    "gaussian_min_max": {
        "func": gaussian_min_max,
        "bounds": [(-1, 1)] * 2
    },
    "giunta": {
        "func": giunta,
        "bounds": [(-1, 1)] * 2
    },
    "goldsteinprice": {
        "func": goldsteinprice,
        "bounds": [(-2, 2)] * 2
    },
    "griewank": {
        "func": griewank,
        "bounds": [(-100, 100)] * 2
    },
    "helicalvally_min_max": {
        "func": helicalvally_min_max,
        "bounds": [(-5, 5)] * 2
    },
    "himmelblau": {
        "func": himmelblau,
        "bounds": [(-5, 5)] * 2
    },
    "holdertable": {
        "func": holdertable,
        "bounds": [(-10, 10)] * 2
    },
    "leon": {
        "func": leon,
        "bounds": [(-1.2, 1.2)] * 2
    },
    "levi13": {
        "func": levi13,
        "bounds": [(-10, 10)] * 2
    },
    "logarithmic_min_max": {
        "func": logarithmic_min_max,
        "bounds": [(1, 2)] * 2
    },
    "matyas": {
        "func": matyas,
        "bounds": [(-10, 10)] * 2
    },
    "mccormick": {
        "func": mccormick,
        "bounds": [(-1.5, 4), (-3, 4)]
    },
    "modschaffer1": {
        "func": modschaffer1,
        "bounds": [(-100, 100)] * 2
    },
    "modschaffer2": {
        "func": modschaffer2,
        "bounds": [(-100, 100)] * 2
    },
    "modschaffer3": {
        "func": modschaffer3,
        "bounds": [(-100, 100)] * 2
    },
    "modschaffer4": {
        "func": modschaffer4,
        "bounds": [(-100, 100)] * 2
    },
    "penholder": {
        "func": penholder,
        "bounds": [(-11, 11)] * 2
    },
    "perlin": {
        "func": perlin,
        "bounds": [(-1, 1)] * 2
    },
    "rastrigin": {
        "func": rastrigin,
        "bounds": [(-5.12, 5.12)] * 2
    },
    "rosenbrock": {
        "func": rosenbrock,
        "bounds": [(-100, 100)] * 2
    },
    "schweffel": {
        "func": schweffel,
        "bounds": [(-500, 500)] * 2
    },
    "sinenvsin": {
        "func": sinenvsin,
        "bounds": [(-100, 100)] * 2
    },
    "sixhumpcamel": {
        "func": sixhumpcamel,
        "bounds": [(-5, 5)] * 2
    },
    "testtubeholder": {
        "func": testtubeholder,
        "bounds": [(-10, 10)] * 2
    },
    "threehumpcamel": {
        "func": threehumpcamel,
        "bounds": [(-5, 5)] * 2
    },
    "zettl": {
        "func": zettl,
        "bounds": [(-5, 5)] * 2
    }
}

In [120]:
obj = list(objectives.keys())[-1] if obj is None else obj

xy_range = objectives[obj]["bounds"]

X = np.arange(*xy_range[0], ((xy_range[0][1] - xy_range[0][0]) / 100))
Y = np.arange(*xy_range[1], ((xy_range[1][1] - xy_range[1][0]) / 100))
X, Y = np.meshgrid(X, Y)
Z = objectives[obj]["func"](X, Y)

z_range = (np.floor(np.min(Z)) - 1, np.ceil(np.max(Z)) + 1)

In [121]:
ran_err = lambda n, x: np.random.normal(0, x, n)
sys_err = lambda n, x, y, poly_model: poly_model.intercept_ + poly_model.coef_[0] * x + poly_model.coef_[1] * y + poly_model.coef_[2] * x**2 + poly_model.coef_[3] * y**2 + poly_model.coef_[4] * x * y

In [122]:
warnings.simplefilter('ignore')

df = pd.DataFrame(np.random.randint(100, size=(1000, 2)))

df.columns = ["i", "j"]

df["x"] = X[0, df["i"]]
df["y"] = Y[df["j"], 0]
df["z"] = Z[df["i"], df["j"]] + ran_err(1000, 0.05)

simul_df = df.copy()

poly_features = PolynomialFeatures(degree=degree, include_bias=False)
poly_features = poly_features.fit_transform(simul_df[["x", "y"]])

poly_model = LinearRegression()
poly_model = poly_model.fit(poly_features, simul_df["z"])

df = pd.DataFrame(np.random.randint(100, size=(m, 2)))

df.columns = ["i", "j"]

df["x"] = X[0, df["i"]]
df["y"] = Y[df["j"], 0]
df["z"] = Z[df["i"], df["j"]] + ran_err(m, 0.05)

for idx in range(1, iterations + 1):
    krnl = RBF(length_scale=1)
    model = GaussianProcessRegressor(kernel=krnl, normalize_y=False, random_state=3, alpha=0.001)

    tmp_df = pd.DataFrame(np.random.randint(100, size=(n, 2)))

    tmp_df.columns = ["i", "j"]

    tmp_df["x"] = X[0, tmp_df["i"]]
    tmp_df["y"] = Y[tmp_df["j"], 0]
    tmp_df["z"] = sys_err(n, tmp_df["x"], tmp_df["y"], poly_model)

    model.fit(tmp_df[["x", "y"]], tmp_df["z"])

    pred = model.predict(tmp_df[["x", "y"]])

    z = griddata((tmp_df["x"], tmp_df["y"]), pred, (X.T, Y.T), method="linear", fill_value=z_range[0])

    tmp_df = pd.DataFrame(list(product(range(100), range(100))), columns=["i", "j"])

    tmp_df["x"] = X[0, tmp_df["i"]]
    tmp_df["y"] = Y[tmp_df["j"], 0]
    tmp_df["z"] = Z[tmp_df["i"], tmp_df["j"]] + ran_err(10000, 0.05)

    mag = z[tmp_df["i"], tmp_df["j"]]

    p = mag - z.min() + laplace_alpha

    p = np.where(p > np.percentile(p, percentile), p, 0)

    p /= p.sum()

    tmp_df = tmp_df.loc[np.random.choice(tmp_df.index, size=m, p=p, replace=False)]

    df = pd.concat([df, tmp_df], ignore_index=True)

    simul_df = pd.concat([simul_df, tmp_df], ignore_index=True)

    poly_features = PolynomialFeatures(degree=degree, include_bias=False)
    poly_features = poly_features.fit_transform(simul_df[["x", "y"]])

    poly_model = LinearRegression()
    poly_model = poly_model.fit(poly_features, simul_df["z"])

krnl = RBF(length_scale=1)
model = GaussianProcessRegressor(kernel=krnl, normalize_y=False, random_state=3, alpha=0.001)

model.fit(df[["x", "y"]], df["z"])

pred = model.predict(df[["x", "y"]])

z = griddata((df["x"], df["y"]), pred, (X.T, Y.T), method="linear", fill_value=(z_range[0] + 1))

print(f"{obj}:", percentileofscore(Z.flatten(), Z[np.unravel_index(z.argmax(), z.shape)]))

crosslegtable: 97.665


In [123]:
warnings.simplefilter('ignore')

for obj in list(objectives.keys()):
    xy_range = objectives[obj]["bounds"]

    X = np.arange(*xy_range[0], ((xy_range[0][1] - xy_range[0][0]) / 100))
    Y = np.arange(*xy_range[1], ((xy_range[1][1] - xy_range[1][0]) / 100))
    X, Y = np.meshgrid(X, Y)
    Z = objectives[obj]["func"](X, Y)

    z_range = (np.floor(np.min(Z)) - 1, np.ceil(np.max(Z)) + 1)

    df = pd.DataFrame(np.random.randint(100, size=(1000, 2)))

    df.columns = ["i", "j"]

    df["x"] = X[0, df["i"]]
    df["y"] = Y[df["j"], 0]
    df["z"] = Z[df["i"], df["j"]] + ran_err(1000, 0.05)

    simul_df = df.copy()

    poly_features = PolynomialFeatures(degree=degree, include_bias=False)
    poly_features = poly_features.fit_transform(simul_df[["x", "y"]])

    poly_model = LinearRegression()
    poly_model = poly_model.fit(poly_features, simul_df["z"])

    df = pd.DataFrame(np.random.randint(100, size=(m, 2)))

    df.columns = ["i", "j"]

    df["x"] = X[0, df["i"]]
    df["y"] = Y[df["j"], 0]
    df["z"] = Z[df["i"], df["j"]] + ran_err(m, 0.05)

    for idx in range(1, iterations + 1):
        krnl = RBF(length_scale=1)
        model = GaussianProcessRegressor(kernel=krnl, normalize_y=False, random_state=3, alpha=0.001)

        tmp_df = pd.DataFrame(np.random.randint(100, size=(n, 2)))

        tmp_df.columns = ["i", "j"]

        tmp_df["x"] = X[0, tmp_df["i"]]
        tmp_df["y"] = Y[tmp_df["j"], 0]
        tmp_df["z"] = sys_err(n, tmp_df["x"], tmp_df["y"], poly_model)

        model.fit(tmp_df[["x", "y"]], tmp_df["z"])

        pred = model.predict(tmp_df[["x", "y"]])

        z = griddata((tmp_df["x"], tmp_df["y"]), pred, (X.T, Y.T), method="linear", fill_value=z_range[0])

        tmp_df = pd.DataFrame(list(product(range(100), range(100))), columns=["i", "j"])

        tmp_df["x"] = X[0, tmp_df["i"]]
        tmp_df["y"] = Y[tmp_df["j"], 0]
        tmp_df["z"] = Z[tmp_df["i"], tmp_df["j"]] + ran_err(10000, 0.05)

        mag = z[tmp_df["i"], tmp_df["j"]]

        p = mag - z.min() + laplace_alpha

        p = np.where(p > np.percentile(p, percentile), p, 0)

        p /= p.sum()

        tmp_df = tmp_df.loc[np.random.choice(tmp_df.index, size=m, p=p, replace=False)]

        df = pd.concat([df, tmp_df], ignore_index=True)

        simul_df = pd.concat([simul_df, tmp_df], ignore_index=True)

        poly_features = PolynomialFeatures(degree=degree, include_bias=False)
        poly_features = poly_features.fit_transform(simul_df[["x", "y"]])

        poly_model = LinearRegression()
        poly_model = poly_model.fit(poly_features, simul_df["z"])

    krnl = RBF(length_scale=1)
    model = GaussianProcessRegressor(kernel=krnl, normalize_y=False, random_state=3, alpha=0.001)

    model.fit(df[["x", "y"]], df["z"])

    pred = model.predict(df[["x", "y"]])

    z = griddata((df["x"], df["y"]), pred, (X.T, Y.T), method="linear", fill_value=(z_range[0] + 1))

    print(f"{obj}:", percentileofscore(Z.flatten(), Z[np.unravel_index(z.argmax(), z.shape)]))

ackley: 99.385
beale: 96.56
bird: 98.58
booth: 98.31
bukin4: 99.57000000000001
bukin6: 99.38
carromtable: 94.08500000000001
chichinadze: 99.37
crossfunc: 38.215
crossintray: 98.735
crosslegtable: 58.315
crownedcross: 97.465
cube: 97.505
easom: 50.035000000000004
eggholder: 99.505
gaussian_min_max: 99.255
giunta: 96.505
goldsteinprice: 99.38
griewank: 99.635
helicalvally_min_max: 98.86
himmelblau: 100.0
holdertable: 94.98
leon: 98.37
levi13: 99.75
logarithmic_min_max: 86.54
matyas: 93.745
mccormick: 98.89
modschaffer1: 59.895
modschaffer2: 99.815
modschaffer3: 62.805
modschaffer4: 66.965
penholder: 99.495
perlin: 98.54
rastrigin: 99.905
rosenbrock: 98.505
schweffel: 98.555
sinenvsin: 61.005
sixhumpcamel: 99.3
testtubeholder: 99.97
threehumpcamel: 99.77
zettl: 97.44
